# Лабораторная 2

In [16]:
from abc import ABCMeta, abstractmethod
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("data\creditcard.csv")

In [7]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [8]:
class Classificator():
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def train(X, Y):
        '''Обучить модель'''
    
    @abstractmethod
    def predict(self, X):
        '''Предсказать данные'''

In [9]:
class DecisionTree(Classificator):
    model = None
    
    def __init__():
        pass
    
    def train(X, Y, prediction, max_depth, leafValue):
        entropy = calculate_entropy(Y)
    
    def predict(self, X):
        pass
    
    def calculate_entropy(Y):
        count = len(df.Class)
        entropy = 0
        for count_class in df.Class.value_counts():
            p = count_class / count
            entropy -= p * np.log2(p)
        return entropy
    
    
    def information_gain():
        pass

In [10]:
class RandomForest(Classificator):
    model = None
    
    def __init__():
        pass
    
    def train(X, Y):
        pass
    
    def predict(self, X):
        pass

In [11]:
class GradientBoosting(Classificator):
    model = None
    
    def __init__():
        pass
    
    def train(X, Y):
        pass
    
    def predict(self, X):
        pass

In [34]:

entropy

0.002490081061839148
0.018343407709838624


0.018343407709838624

In [25]:
len(df.Class)

284807

In [35]:
df.Class.value_counts()[0] / len(df.Class) * np.log2(df.Class.value_counts()[0] / len(df.Class))

-0.002490081061839148

In [36]:
df.Class.value_counts()[1] / len(df.Class) * np.log2(df.Class.value_counts()[1] / len(df.Class))

-0.015853326647999478

In [39]:
df.Class.value_counts()[0] / len(df.Class) + df.Class.value_counts()[1] / len(df.Class)

1.0

In [45]:
- (4/20) * np.log2(4 / 20) - (16/20) * np.log2(16 / 20)

0.7219280948873623

9.179604964549883